# Update Production Data and Model

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-12-24  

This file contains update codes for the production server. The update should be scheduled every eight weeks.

## 1. Import packages

In [3]:
import numpy as np
import pandas as pd

import os
import re
import time
import datetime
import sys
from tqdm import tqdm

import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import funclib
import exact_ec_from_uniprot as exactec
import minitools as mtool
import embedding_esm as esmebd
# import embedding_unirep as unirep

from pandarallel import pandarallel 
pandarallel.initialize() 
import benchmark_train as btrain

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Define Functions

In [4]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel -y

# add missing '-' for ec number
def refill_ec(ec):   
    if ec == '-':
        return ec
    levelArray = ec.split('.')
    if  levelArray[3]=='':
        levelArray[3] ='-'
    ec = '.'.join(levelArray)
    return ec

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecs = ecstr.split(',')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

#format ec
def format_ec(ecstr):
    ecArray= ecstr.split(',')
    ecArray=[x.strip() for x in ecArray] #strip blank
    ecArray=[refill_ec(x) for x in ecArray] #format ec to full
    ecArray = list(set(ecArray)) # remove duplicates
    
    return ','.join(ecArray)

## 3. Download latest data from unisprot

In [ ]:
# download location ./tmp

! mv $cfg.DATADIR'uniprot_sprot_latest.dat.gz' $cfg.TEMPDIR$currenttime'_uniprot_sprot_latest.dat.gz'
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz -o ./data/uniprot_sprot_latest.dat.gz -q -c

## 4. Preprocessing

In [9]:
exactec.run_exact_task(infile=cfg.DATADIR+'uniprot_sprot_latest.dat.gz', outfile=cfg.DATADIR+'sprot_latest.tsv')

#加载数据并转换时间格式
sprot_latest = pd.read_csv(cfg.DATADIR+'sprot_latest.tsv', sep='\t',header=0) #读入文件
sprot_latest = mtool.convert_DF_dateTime(inputdf = sprot_latest)

sprot_latest.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot_latest.reset_index(drop=True, inplace=True)

#sprot_latest format EC
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: format_ec(x))
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot_latest['functionCounts'] = sprot_latest.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

# Trim Strging
with pd.option_context('mode.chained_assignment', None):
    sprot_latest.ec_number = sprot_latest.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    sprot_latest.seq = sprot_latest.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

sprot_latest.to_feather(cfg.DATADIR + 'latest_sprot.feather')


## 5. Caculation Features

In [17]:
train= pd.read_feather(cfg.DATADIR + 'latest_sprot.feather')
print('train size: {0}'.format(len(train)))

train size: 477917


In [ ]:
! mv $cfg.DATADIR'sprot_latest_rep0.feather' $cfg.DATADIR'featureBank/sprot_latest_rep0.feather'
! mv $cfg.DATADIR'sprot_latest_rep32.feather' $cfg.DATADIR'featureBank/sprot_latest_rep32.feather'
! mv $cfg.DATADIR'sprot_latest_rep33.feather' $cfg.DATADIR'featureBank/sprot_latest_rep33.feather'
! mv $cfg.DATADIR'sprot_latest_unirep.feather' $cfg.DATADIR'featureBank/sprot_latest_unirep.feather'

# !pip install fair-esm
tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=train, model='esm1b_t33_650M_UR50S',seqthres=1022)
tr_rep0.to_feather(cfg.DATADIR + 'sprot_latest_rep0.feather')
tr_rep32.to_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
tr_rep33.to_feather(cfg.DATADIR + 'sprot_latest_rep33.feather')



In [19]:
train_esm_latest = pd.read_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
train_esm_latest = train.merge(train_esm_latest, on='id', how='left')

## 6. Split X Y

In [33]:
# task 1
X_train_task1 = np.array(train_esm_latest.iloc[:,12:])
Y_train_task1 = np.array(train_esm_latest.isenzyme.astype('int')).flatten()
train_enzyme = train_esm_latest[train_esm_latest.isenzyme].reset_index(drop=True)

# task 2
X_train_task2_s = np.array(train_enzyme.iloc[:,12:])
Y_train_task2_s = train_enzyme.functionCounts.apply(lambda x : 0 if x==1 else 1).astype('int').values

train_task2M=train_enzyme[train_enzyme.functionCounts>=2].reset_index(drop=True)
X_train_task2_m = np.array(train_task2M.iloc[:,12:])
Y_train_task2_m = np.array(train_task2M.functionCounts.astype('int')-2).flatten()

#task 3
train_set_task3= funclib.split_ecdf_to_single_lines(train_enzyme.iloc[:,np.r_[0,10,5]])
train_set_task3=train_set_task3.merge(train_esm_latest.iloc[:,np.r_[0,12:1292]], on='id', how='left')

#4. 加载EC号训练数据
print('loading ec to label dict')
dict_ec_label = btrain.make_ec_label(train_label=train_set_task3['ec_number'], test_label=train_set_task3['ec_number'], file_save= cfg.FILE_EC_LABEL_DICT, force_model_update=cfg.UPDATE_MODEL)

train_set_task3['ec_label']=train_set_task3.ec_number.parallel_apply(lambda x: dict_ec_label.get(x))    
X_train_task3 = np.array(train_set_task3.iloc[:,3:])
Y_train_task3 = np.array(train_set_task3.ec_label.astype('int')).flatten()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 231373/231373 [04:50<00:00, 797.26it/s]


loading ec to label dict


## 7. Train Model

In [ ]:
# Task 1
groundtruth, predict, predictprob, model = funclib.knnmain(X_train_task1, Y_train_task1, X_train_task1.iloc[0:10,:], Y_train_task1.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.ISENZYME_MODEL)

In [ ]:
groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_s, Y_train_task2_s, X_train_task2_s.iloc[0:10,:], Y_train_task2_s.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.MODELDIR+'/single_multi.model')

groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_m, Y_train_task2_m, X_train_task2_m.iloc[0:10,:], Y_train_task2_m.iloc[0:10,:], type='multi')
joblib.dump(model, cfg.MODELDIR+'/multi_many.model')

In [ ]:
cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=X_train_task3, y_data=Y_train_task3, x_file=cfg.DATADIR+'slice_train_x_esm32_latest.txt', y_file=cfg.DATADIR+'slice_train_y_esm32_latest.txt', ec_label_dict=dict_ec_label)
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm32_latest.txt', trainY=cfg.DATADIR+'slice_train_y_esm32_latest.txt', modelPath=cfg.MODELDIR+'/slice_esm32')

In [129]:
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER)

In [66]:
bb=pd.read_csv('./tmp/ecolir.tsv', sep='\t')

In [77]:
bb

,id,id_uniprot,seq,seqlength,date_integraged,date_sequence_update,date_annotation_update,res_type,top0,top1,...,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19
0,b0001,P0AD87,MKRISTTITTTITITTGNGAG,21,1986-07-21,1986-07-21,2021-09-29,db_match,-,2.7.7.61,...,1.11.1.16,2.7.1.87,3.1.4.4,2.7.1.169,3.1.22.-,2.7.11.11,3.4.23.2,3.5.4.4,2.7.1.30,3.5.4.30
1,b0002,P00561,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,820,1986-07-21,2003-08-29,2021-09-29,db_match,"2.7.2.4,1.1.1.3",2.7.1.176,...,1.2.1.-,4.2.3.36,1.1.1.81,1.8.2.3,3.4.24.1,4.2.3.115,1.3.7.12,1.1.1.212,1.1.1.392,4.2.3.90
2,b0003,P00547,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,310,1986-07-21,1993-04-01,2020-12-02,db_match,2.7.1.39,2.6.1.52,...,2.5.1.23,2.8.4.2,2.6.1.62,2.7.8.37,2.5.1.76,2.6.1.86,4.2.3.195,2.5.1.85,1.1.5.9,2.6.1.96
3,b0004,P00934,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...,428,1986-07-21,1986-07-21,2021-06-02,db_match,4.2.3.1,4.1.1.77,...,1.1.1.372,3.1.2.-,4.2.3.115,2.4.1.92,2.7.11.12,4.2.3.113,4.2.1.50,4.2.3.15,4.2.3.22,3.4.24.53
4,b0005,P75616,MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...,98,2000-05-30,1997-02-01,2021-06-02,db_match,-,2.3.1.169,...,1.7.1.13,2.7.8.32,1.1.1.88,3.1.2.-,4.1.3.-,2.8.2.21,1.1.1.9,3.4.17.13,2.1.1.303,3.4.21.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,b4706,U3PVA8,MKPALRDFIAIVQERLASVTA,21,2014-06-11,2013-12-11,2020-10-07,blast_match,-,2.7.7.27,...,1.1.1.56,1.3.8.6,2.7.9.3,1.2.1.79,3.1.7.12,3.2.1.31,4.1.3.6,1.2.1.64,3.2.1.209,2.3.1.217
4318,b4708,P0CF88,ENTNGLIRQYFPKKTCLAQYTQHELDLVAAQLNNRPRKTLKFKTPK...,57,2010-05-18,2010-05-18,2021-09-29,blast_match,-,2.7.1.43,...,2.6.1.39,2.1.1.288,4.2.3.100,1.7.99.1,1.8.1.14,3.2.1.93,3.4.25.-,5.5.1.27,2.1.1.106,2.7.1.130
4319,b4709,P0ADH1,MASVSISCPSCSATDGVVRNGKSTAGHQRYLCSHCRKTWQLQFTYT...,91,1986-07-21,2005-12-06,2021-04-07,blast_match,-,2.7.7.27,...,3.1.22.-,2.8.1.15,4.1.2.8,5.5.1.27,3.1.4.2,2.7.1.167,2.7.1.169,5.1.99.6,1.2.7.4,3.1.3.78
4320,b4710,P59843,MPGNSPHYGRWPQHDFTSLKKLRPQSVTSRIQPGSDVIVCAEMDEQ...,167,2003-09-09,2003-09-09,2020-12-02,blast_match,-,2.7.1.52,...,3.4.24.59,3.5.1.4,5.1.99.7,5.5.1.27,2.8.2.1,2.7.8.33,1.15.1.2,5.3.2.5,4.1.3.-,3.1.3.78


In [68]:
with pd.option_context('mode.chained_assignment', None): 
    bb[bb.id.isin(bb.iloc[0:10,0])]['res_type']='db_match'

In [74]:
bb[bb.id.isin(bb.iloc[0:10,0])].index.values

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [76]:
bb.loc[bb[bb.id.isin(bb.iloc[0:10,0])].index.values,'res_type']= 'db_match'